In [1]:
# CẬP NHẬT THƯ VIỆN LÊN PHIÊN BẢN MỚI NHẤT để khắc phục tất cả lỗi import
!pip install vnstock --upgrade --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
# Chỉ import các module cơ bản và Company
from vnstock import Listing, Quote, Company 
from datetime import datetime

# --- CẤU HÌNH ---
DATA_SOURCE = 'VCI' 
INDEX_SYMBOL = 'VNINDEX'
START_DATE = '2018-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
NUM_STOCKS = 10

# Khởi tạo các danh sách chứa dữ liệu (Bắt buộc phải chạy lại)
all_prices_data = []
vnindex_data = []
all_company_reports = [] # FIX LỖI: name 'all_company_reports' is not defined
all_events_data = [] # Sẽ để trống hoặc dùng cho dữ liệu Officers nếu cần

print(f"Nguồn dữ liệu sử dụng: {DATA_SOURCE}")
print(f"Ngày bắt đầu thu thập: {START_DATE}")

Nguồn dữ liệu sử dụng: VCI
Ngày bắt đầu thu thập: 2018-01-01


In [3]:
print("Bắt đầu lấy danh sách 10 mã cổ phiếu đầu tiên từ VCI...")

try:
    listing = Listing(source=DATA_SOURCE)
    top_symbols = listing.all_symbols()['symbol'].head(NUM_STOCKS).tolist()
    
    # Kiểm tra và loại bỏ VNINDEX nếu nó lọt vào danh sách mã cổ phiếu
    if INDEX_SYMBOL in top_symbols:
        top_symbols.remove(INDEX_SYMBOL)
        
    print(f"✅ Đã xác định thành công {len(top_symbols)} mã: {top_symbols}")

except Exception as e:
    print(f"❌ Lỗi khi lấy danh sách mã cổ phiếu từ {DATA_SOURCE}: {e}")
    # Sử dụng danh sách mặc định các mã lớn nếu lỗi
    top_symbols = ['VCB', 'BID', 'CTG', 'HDB', 'HPG', 'FPT', 'MSN', 'GAS', 'MWG', 'VNM']
    print(f"Sử dụng danh sách mã mặc định để tiếp tục: {top_symbols}")

Bắt đầu lấy danh sách 10 mã cổ phiếu đầu tiên từ VCI...
✅ Đã xác định thành công 10 mã: ['YTC', 'YEG', 'YBM', 'YBC', 'XPH', 'XMP', 'XMD', 'XMC', 'XLV', 'XHC']


In [4]:
import time
from vnstock import Quote # Đảm bảo đã import Quote ở Cell 2

print("\n--- BẮT ĐẦU THU THẬP DỮ LIỆU LỊCH SỬ GIÁ CỔ PHIẾU (Đã thêm cột Open) ---")

DATA_SOURCE = 'VCI'
DATE_COL_OPTIONS = ['TradingDate', 'time'] 
CLOSE_COL_OPTIONS = ['Close', 'close']
VOLUME_COL_OPTIONS = ['Volume', 'volume']
# THÊM TÙY CHỌN CỘT OPEN
OPEN_COL_OPTIONS = ['Open', 'open', 'o'] # Tùy chọn tên cột Open

def find_column(df_columns, options):
    """Tìm tên cột đúng trong DataFrame dựa trên danh sách tùy chọn"""
    for col in options:
        if col in df_columns:
            return col
    return None

# --- LẤY GIÁ CHO CHỈ SỐ VNINDEX (Dùng cú pháp tương tự) ---
try:
    print(f"➡️ Đang tải giá lịch sử cho: {INDEX_SYMBOL}...", end=' ')
    quote_index = Quote(symbol=INDEX_SYMBOL, source=DATA_SOURCE)
    df_index = quote_index.history(start=START_DATE, end=END_DATE)
    
    if not df_index.empty:
        date_col_name = find_column(df_index.columns, DATE_COL_OPTIONS)
        close_col_name = find_column(df_index.columns, CLOSE_COL_OPTIONS)
        open_col_name = find_column(df_index.columns, OPEN_COL_OPTIONS) # Tìm cột Open
        volume_col_name = find_column(df_index.columns, VOLUME_COL_OPTIONS)
        
        # Chỉ kiểm tra Date, Close, Volume
        if date_col_name and close_col_name and volume_col_name:
            df_index['Symbol'] = INDEX_SYMBOL
            
            # Ánh xạ cột
            rename_dict = {date_col_name: 'Date', close_col_name: 'Close', volume_col_name: 'Volume'}
            if open_col_name:
                 rename_dict[open_col_name] = 'Open' # Thêm cột Open nếu tìm thấy
            
            df_index = df_index.rename(columns=rename_dict)
            
            # Giữ lại các cột quan trọng (Thêm Open)
            cols_to_keep = ['Symbol', 'Date', 'Close', 'Volume']
            if 'Open' in df_index.columns:
                 cols_to_keep.insert(2, 'Open') 

            df_index = df_index[cols_to_keep]
            vnindex_data.append(df_index)
            print(f"✅ Hoàn tất. ({len(df_index)} dòng)")
        else:
             print("\n❌ Lỗi: Dữ liệu VNINDEX thiếu cột cần thiết. Bỏ qua.")
    else:
        print(f"⚠️ Không có dữ liệu giá cho VNINDEX.")
except Exception as e:
    print(f"❌ Lỗi khi tải giá cho VNINDEX: {e}")

# --- LẤY GIÁ CHO CÁC MÃ CỔ PHIẾU ---
for symbol in top_symbols:
    print(f"➡️ Đang tải giá lịch sử cho: {symbol}...", end=' ')
    
    try:
        quote = Quote(symbol=symbol, source=DATA_SOURCE)
        df = quote.history(start=START_DATE, end=END_DATE)

        if df.empty:
            print(f"⚠️ Không có dữ liệu giá.")
            continue
            
        # 1. Tìm tên cột chính xác
        date_col_name = find_column(df.columns, DATE_COL_OPTIONS)
        close_col_name = find_column(df.columns, CLOSE_COL_OPTIONS)
        volume_col_name = find_column(df.columns, VOLUME_COL_OPTIONS)
        open_col_name = find_column(df.columns, OPEN_COL_OPTIONS) # TÌM CỘT OPEN
        
        # 2. KIỂM TRA TÍNH HỢP LỆ VÀ BỎ QUA MÃ BỊ THIẾU CỘT
        if not (date_col_name and close_col_name and volume_col_name):
              print(f"\n❌ Lỗi: Dữ liệu VCI cho {symbol} thiếu cột cần thiết. Bỏ qua mã này.")
              continue
              
        # 3. Ánh xạ và chuẩn hóa tên cột
        df['Symbol'] = symbol
        rename_dict = {date_col_name: 'Date', close_col_name: 'Close', volume_col_name: 'Volume'}
        
        # THÊM CỘT OPEN NẾU TÌM THẤY
        if open_col_name:
            rename_dict[open_col_name] = 'Open'
            
        df = df.rename(columns=rename_dict)
        
        # 4. Giữ lại các cột quan trọng (THÊM OPEN VÀO DANH SÁCH)
        cols_to_keep = ['Symbol', 'Date', 'Close', 'Volume']
        if 'Open' in df.columns:
            cols_to_keep.insert(2, 'Open') # Chèn Open vào vị trí thứ 3

        df = df[cols_to_keep] 
        all_prices_data.append(df)
        
        print(f"✅ Hoàn tất. ({len(df)} dòng)")
        time.sleep(0.5) # Độ trễ để tránh rate limit
    except Exception as e:
        print(f"❌ Lỗi khi tải giá cho {symbol}: {e}")

print("\n--- KẾT THÚC THU THẬP DỮ LIỆU LỊCH SỬ GIÁ CỔ PHIẾU ---")


--- BẮT ĐẦU THU THẬP DỮ LIỆU LỊCH SỬ GIÁ CỔ PHIẾU (Đã thêm cột Open) ---
➡️ Đang tải giá lịch sử cho: VNINDEX... ✅ Hoàn tất. (2065 dòng)
➡️ Đang tải giá lịch sử cho: YTC... ✅ Hoàn tất. (2015 dòng)
➡️ Đang tải giá lịch sử cho: YEG... ✅ Hoàn tất. (1857 dòng)
➡️ Đang tải giá lịch sử cho: YBM... ✅ Hoàn tất. (1750 dòng)
➡️ Đang tải giá lịch sử cho: YBC... ✅ Hoàn tất. (2065 dòng)
➡️ Đang tải giá lịch sử cho: XPH... ✅ Hoàn tất. (2065 dòng)
➡️ Đang tải giá lịch sử cho: XMP... ✅ Hoàn tất. (1055 dòng)
➡️ Đang tải giá lịch sử cho: XMD... ✅ Hoàn tất. (2065 dòng)
➡️ Đang tải giá lịch sử cho: XMC... ✅ Hoàn tất. (2065 dòng)
➡️ Đang tải giá lịch sử cho: XLV... ✅ Hoàn tất. (1786 dòng)
➡️ Đang tải giá lịch sử cho: XHC... ✅ Hoàn tất. (2065 dòng)

--- KẾT THÚC THU THẬP DỮ LIỆU LỊCH SỬ GIÁ CỔ PHIẾU ---


In [5]:
print(f"\n--- BẮT ĐẦU THU THẬP DỮ LIỆU LỊCH SỬ CHỈ SỐ: {INDEX_SYMBOL} ---")

try:
    quote = Quote(symbol=INDEX_SYMBOL, source=DATA_SOURCE)
    df = quote.history(start=START_DATE, end=END_DATE)
    
    if df.empty:
        print(f"⚠️ Không có dữ liệu cho {INDEX_SYMBOL}.")
    else:
        # Chuẩn hóa tên cột tương tự Cell 4
        date_col_name = find_column(df.columns, DATE_COL_OPTIONS)
        close_col_name = find_column(df.columns, CLOSE_COL_OPTIONS)
        
        df['Symbol'] = INDEX_SYMBOL
        df = df.rename(columns={date_col_name: 'Date', 
                                close_col_name: 'Close'})
        
        # Chỉ giữ lại Ngày và Giá đóng cửa
        vnindex_data.append(df[['Symbol', 'Date', 'Close']])
        
        print(f"✅ Hoàn tất tải {INDEX_SYMBOL}. ({len(df)} dòng)")

except Exception as e:
    print(f"❌ Lỗi khi tải {INDEX_SYMBOL}: {e}")


--- BẮT ĐẦU THU THẬP DỮ LIỆU LỊCH SỬ CHỈ SỐ: VNINDEX ---
✅ Hoàn tất tải VNINDEX. (2065 dòng)


In [6]:
import time

# Tách danh sách top_symbols thành hai phần
split_point = len(top_symbols) // 2
top_symbols_part1 = top_symbols[:split_point]
top_symbols_part2 = top_symbols[split_point:]

print(f"Tổng số mã: {len(top_symbols)}")
print(f"Phần 1: {top_symbols_part1}")
print(f"Phần 2: {top_symbols_part2}")

Tổng số mã: 10
Phần 1: ['YTC', 'YEG', 'YBM', 'YBC', 'XPH']
Phần 2: ['XMP', 'XMD', 'XMC', 'XLV', 'XHC']


In [7]:
print("\n--- BẮT ĐẦU THU THẬP DỮ LIỆU CÔNG TY: PHẦN 1 ---")

for symbol in top_symbols_part1:
    print(f"➡️ Đang tải dữ liệu cho: {symbol}...")
    
    # BƯỚC 1: TẠO ĐỐI TƯỢNG COMPANY
    try:
        company = Company(symbol=symbol, source=DATA_SOURCE)
    except Exception as e:
        print(f"   - ❌ Lỗi khi khởi tạo Company cho {symbol}: {e}. Bỏ qua mã này.")
        continue

    # 2. Ratio Summary - company.ratio_summary()
    try:
        indicators_df = company.ratio_summary() 
        indicators_df['Report_Type'] = 'RatioSummary'
        indicators_df['Symbol'] = symbol
        all_company_reports.append(indicators_df)
        print(f"   - ✅ Ratio Summary (Chỉ số): {len(indicators_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Ratio Summary: {e}")
        
    # 3. Report - company.reports() (Khối thử nghiệm 0 dòng)
    try:
        # Thử gọi với tham số report_type để khắc phục lỗi 0 dòng (lần thử thứ nhất)
        statements_df = company.reports(report_type='IncomeStatement')
        statements_df['Report_Type'] = 'Reports_IS'
        statements_df['Symbol'] = symbol
        all_company_reports.append(statements_df)
        print(f"   - ✅ Report (Báo cáo KQKD): {len(statements_df)} dòng")
    except Exception as e:
        # Nếu thất bại, quay lại cú pháp gốc (không tham số) (lần thử thứ hai)
        try:
            statements_df = company.reports() 
            statements_df['Report_Type'] = 'Reports_Default'
            statements_df['Symbol'] = symbol
            all_company_reports.append(statements_df)
            print(f"   - ✅ Report (Báo cáo KQKD - Mặc định): {len(statements_df)} dòng")
        except Exception as fallback_e:
             print(f"   - ❌ Lỗi khi tải Report: {fallback_e}")
        
    # 4. Cán bộ/Quản lý - company.officers(filter_by='working')
    try:
        officers_df = company.officers(filter_by='working') 
        officers_df['Report_Type'] = 'Officers'
        officers_df['Symbol'] = symbol
        all_company_reports.append(officers_df)
        print(f"   - ✅ Officers (Cán bộ): {len(officers_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Officers: {e}")
        
    # 5. Công ty con (Subsidiaries) - company.subsidiaries()
    try:
        subs_df = company.subsidiaries() 
        subs_df['Report_Type'] = 'Subsidiaries'
        subs_df['Symbol'] = symbol
        all_company_reports.append(subs_df)
        print(f"   - ✅ Công ty con: {len(subs_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: {e}")
        
    # 6. Tổng quan Công ty (Overview) - company.overview()
    try:
        overview_series = company.overview() 
        overview_df = pd.DataFrame([overview_series.to_dict()])
        overview_df['Report_Type'] = 'Overview'
        overview_df['Symbol'] = symbol
        all_company_reports.append(overview_df)
        print(f"   - ✅ Overview (Tổng quan): 1 dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Overview: {e}")

    # 7. Cổ đông (Shareholders) - company.shareholders()
    try:
        shareholders_df = company.shareholders() 
        shareholders_df['Report_Type'] = 'Shareholders'
        shareholders_df['Symbol'] = symbol
        all_company_reports.append(shareholders_df)
        print(f"   - ✅ Shareholders (Cổ đông): {len(shareholders_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Shareholders: {e}")
        
    # ĐỘ TRỄ NHỎ GIỮA CÁC MÃ
    time.sleep(0.5) 

print("\n--- KẾT THÚC THU THẬP PHẦN 1 ---")


--- BẮT ĐẦU THU THẬP DỮ LIỆU CÔNG TY: PHẦN 1 ---
➡️ Đang tải dữ liệu cho: YTC...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 9 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886210c580 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 6 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: YEG...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 1 dòng
   - ✅ Officers (Cán bộ): 18 dòng
   - ✅ Công ty con: 13 dòng
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 28 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: YBM...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 12 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886254e230 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 16 dòng


...

➡️ Đang tải dữ liệu cho: YBC...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 9 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886254fb50 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 12 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: XPH...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 9 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886253d4b0 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 8 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.



--- KẾT THÚC THU THẬP PHẦN 1 ---


In [8]:
print("\n--- BẮT ĐẦU THU THẬP DỮ LIỆU CÔNG TY: PHẦN 2 ---")

for symbol in top_symbols_part2:
    print(f"➡️ Đang tải dữ liệu cho: {symbol}...")
    
    # BƯỚC 1: TẠO ĐỐI TƯỢNG COMPANY
    try:
        company = Company(symbol=symbol, source=DATA_SOURCE)
    except Exception as e:
        print(f"   - ❌ Lỗi khi khởi tạo Company cho {symbol}: {e}. Bỏ qua mã này.")
        continue

    # 2. Ratio Summary - company.ratio_summary()
    try:
        indicators_df = company.ratio_summary() 
        indicators_df['Report_Type'] = 'RatioSummary'
        indicators_df['Symbol'] = symbol
        all_company_reports.append(indicators_df)
        print(f"   - ✅ Ratio Summary (Chỉ số): {len(indicators_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Ratio Summary: {e}")
        
    # 3. Report - company.reports() (Khối thử nghiệm 0 dòng)
    try:
        # Thử gọi với tham số report_type để khắc phục lỗi 0 dòng (lần thử thứ nhất)
        statements_df = company.reports(report_type='IncomeStatement')
        statements_df['Report_Type'] = 'Reports_IS'
        statements_df['Symbol'] = symbol
        all_company_reports.append(statements_df)
        print(f"   - ✅ Report (Báo cáo KQKD): {len(statements_df)} dòng")
    except Exception as e:
        # Nếu thất bại, quay lại cú pháp gốc (không tham số) (lần thử thứ hai)
        try:
            statements_df = company.reports() 
            statements_df['Report_Type'] = 'Reports_Default'
            statements_df['Symbol'] = symbol
            all_company_reports.append(statements_df)
            print(f"   - ✅ Report (Báo cáo KQKD - Mặc định): {len(statements_df)} dòng")
        except Exception as fallback_e:
             print(f"   - ❌ Lỗi khi tải Report: {fallback_e}")
        
    # 4. Cán bộ/Quản lý - company.officers(filter_by='working')
    try:
        officers_df = company.officers(filter_by='working') 
        officers_df['Report_Type'] = 'Officers'
        officers_df['Symbol'] = symbol
        all_company_reports.append(officers_df)
        print(f"   - ✅ Officers (Cán bộ): {len(officers_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Officers: {e}")
        
    # 5. Công ty con (Subsidiaries) - company.subsidiaries()
    try:
        subs_df = company.subsidiaries() 
        subs_df['Report_Type'] = 'Subsidiaries'
        subs_df['Symbol'] = symbol
        all_company_reports.append(subs_df)
        print(f"   - ✅ Công ty con: {len(subs_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: {e}")
        
    # 6. Tổng quan Công ty (Overview) - company.overview()
    try:
        overview_series = company.overview() 
        overview_df = pd.DataFrame([overview_series.to_dict()])
        overview_df['Report_Type'] = 'Overview'
        overview_df['Symbol'] = symbol
        all_company_reports.append(overview_df)
        print(f"   - ✅ Overview (Tổng quan): 1 dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Overview: {e}")

    # 7. Cổ đông (Shareholders) - company.shareholders()
    try:
        shareholders_df = company.shareholders() 
        shareholders_df['Report_Type'] = 'Shareholders'
        shareholders_df['Symbol'] = symbol
        all_company_reports.append(shareholders_df)
        print(f"   - ✅ Shareholders (Cổ đông): {len(shareholders_df)} dòng")
    except Exception as e:
        print(f"   - ❌ Lỗi khi tải Shareholders: {e}")
        
    # ĐỘ TRỄ NHỎ GIỮA CÁC MÃ
    time.sleep(0.5) 

print("\n--- KẾT THÚC THU THẬP PHẦN 2 ---")


--- BẮT ĐẦU THU THẬP DỮ LIỆU CÔNG TY: PHẦN 2 ---
➡️ Đang tải dữ liệu cho: XMP...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 10 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886210e500 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 6 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: XMD...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 8 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x188620fab00 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 8 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: XMC...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 14 dòng
   - ✅ Công ty con: 14 dòng
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 31 dòng


...

➡️ Đang tải dữ liệu cho: XLV...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 10 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x1886210fd00 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 19 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


➡️ Đang tải dữ liệu cho: XHC...
   - ✅ Ratio Summary (Chỉ số): 1 dòng
   - ✅ Report (Báo cáo KQKD - Mặc định): 0 dòng
   - ✅ Officers (Cán bộ): 11 dòng
   - ❌ Lỗi khi tải Công ty con: Lỗi dữ liệu/API bên trong: RetryError[<Future at 0x188625fd840 state=finished raised KeyError>]
   - ✅ Overview (Tổng quan): 1 dòng
   - ✅ Shareholders (Cổ đông): 20 dòng


DataFrame.applymap has been deprecated. Use DataFrame.map instead.



--- KẾT THÚC THU THẬP PHẦN 2 ---


In [9]:
import pandas as pd
import os # Thêm thư viện os để quản lý file system

print("\n--- BẮT ĐẦU HỢP NHẤT VÀ LƯU TRỮ DỮ LIỆU ---")

# 1. KHỞI TẠO THƯ MỤC LƯU TRỮ
# Sẽ tạo thư mục 'Data' trong thư mục hiện tại của Notebook
OUTPUT_DIR = 'Data' 
os.makedirs(OUTPUT_DIR, exist_ok=True) # Tạo thư mục nếu chưa tồn tại

# --- Hàm lọc chỉ lấy DataFrame hợp lệ ---
def filter_valid_dataframes(data_list):
    """Lọc danh sách để chỉ giữ lại các đối tượng là pandas DataFrame."""
    return [df for df in data_list if isinstance(df, pd.DataFrame)]

# 1. Hợp nhất và Lưu Dữ liệu Giá Cổ phiếu
valid_prices = filter_valid_dataframes(all_prices_data)
if valid_prices:
    final_prices_df = pd.concat(valid_prices, ignore_index=True)
    file_name_prices = f'{DATA_SOURCE}_top_stocks_prices.csv'
    # Lưu vào đường dẫn tương đối: Data/VCI_top_stocks_prices.csv
    output_path = os.path.join(OUTPUT_DIR, file_name_prices) 
    final_prices_df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu Dữ liệu Giá Cổ phiếu ({len(final_prices_df)} dòng) vào: {output_path}")
else:
    print("⚠️ Không có dữ liệu giá cổ phiếu hợp lệ để lưu.")

# 2. Hợp nhất và Lưu Dữ liệu VNINDEX
valid_vnindex = filter_valid_dataframes(vnindex_data)
if valid_vnindex:
    final_vnindex_df = pd.concat(valid_vnindex, ignore_index=True)
    file_name_vnindex = f'{DATA_SOURCE}_VNINDEX_prices.csv'
    output_path = os.path.join(OUTPUT_DIR, file_name_vnindex)
    final_vnindex_df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu Dữ liệu Giá VNINDEX ({len(final_vnindex_df)} dòng) vào: {output_path}")
else:
    print("⚠️ Không có dữ liệu VNINDEX hợp lệ để lưu.")

# 3. Hợp nhất và Lưu TẤT CẢ Dữ liệu Công ty
valid_reports = filter_valid_dataframes(all_company_reports)
if valid_reports:
    final_reports_df = pd.concat(valid_reports, ignore_index=True)
    file_name_reports = f'{DATA_SOURCE}_company_reports_summary.csv'
    output_path = os.path.join(OUTPUT_DIR, file_name_reports)
    final_reports_df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu Dữ liệu Công ty Tổng hợp ({len(final_reports_df)} dòng) vào: {output_path}")
else:
    print("⚠️ Không có dữ liệu báo cáo/chỉ số công ty hợp lệ để lưu.")
    
# 4. Dữ liệu Events (nếu có)
valid_events = filter_valid_dataframes(all_events_data)
if valid_events:
    final_events_df = pd.concat(valid_events, ignore_index=True)
    file_name_events = f'{DATA_SOURCE}_corporate_events.csv'
    output_path = os.path.join(OUTPUT_DIR, file_name_events)
    final_events_df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu Dữ liệu Sự kiện Doanh nghiệp ({len(final_events_df)} dòng) vào: {file_name_events}")
else:
    print("⚠️ Không có dữ liệu sự kiện doanh nghiệp hợp lệ để lưu.")

print("\n--- HOÀN TẤT LƯU TRỮ DỮ LIỆU ---")


--- BẮT ĐẦU HỢP NHẤT VÀ LƯU TRỮ DỮ LIỆU ---
✅ Đã lưu Dữ liệu Giá Cổ phiếu (18788 dòng) vào: Data\VCI_top_stocks_prices.csv
✅ Đã lưu Dữ liệu Giá VNINDEX (4130 dòng) vào: Data\VCI_VNINDEX_prices.csv
✅ Đã lưu Dữ liệu Công ty Tổng hợp (312 dòng) vào: Data\VCI_company_reports_summary.csv
⚠️ Không có dữ liệu sự kiện doanh nghiệp hợp lệ để lưu.

--- HOÀN TẤT LƯU TRỮ DỮ LIỆU ---
